In [9]:
import tensorflow as tf
import tensorflow.keras as keras
from d2l import tensorflow as d2l

class Residual(keras.Model):
    def __init__(self,num_channels,use_1x1conv = False,strides = 1):
        super(Residual, self).__init__()
        self.conv1 = keras.layers.Conv2D(
            num_channels,padding = 'same',kernel_size=3,strides=strides)
        self.conv2 = keras.layers.Conv2D(
            num_channels,padding = 'same',kernel_size=3)
        self.conv3 = None
        if use_1x1conv:
            self.conv3 = keras.layers.Conv2D(
                num_channels,kernel_size=1,strides = strides)
        self.bn1 = keras.layers.BatchNormalization()
        self.bn2 = keras.layers.BatchNormalization()

    def call(self,X):
        Y = keras.activations.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3 is not None:
            X = self.conv3(X)
        Y += X
        return keras.activations.relu(Y)

In [11]:
b1 = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')
    ])

In [13]:
class ResnetBlock(keras.layers.Layer):
    def __init__(self,num_channels,num_residuals,
                 first_block = False,**kwargs):
        super(ResnetBlock, self).__init__()
        self.residual_layers = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                self.residual_layers.append(
                    Residual(num_channels,use_1x1conv=True,strides=2)
                )
            else:
                self.residual_layers.append(Residual(num_channels))

    def call(self,X):
        for layer in self.residual_layers.layers:
            X = layer(X)
        return X

In [14]:
b2 = ResnetBlock(64, 2, first_block=True)
b3 = ResnetBlock(128, 2)
b4 = ResnetBlock(256, 2)
b5 = ResnetBlock(512, 2)

In [16]:
def net():
    return keras.Sequential([

        keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'),

        ResnetBlock(64, 2, first_block=True),
        ResnetBlock(128, 2),
        ResnetBlock(256, 2),
        ResnetBlock(512, 2),
        keras.layers.GlobalAvgPool2D(),
        keras.layers.Dense(units=10)
    ])

In [17]:
X = tf.random.uniform(shape=(1, 224, 224, 1))
for layer in net().layers:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Conv2D output shape:	 (1, 112, 112, 64)
BatchNormalization output shape:	 (1, 112, 112, 64)
Activation output shape:	 (1, 112, 112, 64)
MaxPooling2D output shape:	 (1, 56, 56, 64)
ResnetBlock output shape:	 (1, 56, 56, 64)
ResnetBlock output shape:	 (1, 28, 28, 128)
ResnetBlock output shape:	 (1, 14, 14, 256)
ResnetBlock output shape:	 (1, 7, 7, 512)
GlobalAveragePooling2D output shape:	 (1, 512)
Dense output shape:	 (1, 10)
